In [1]:
import sys
from pathlib import Path
wdir = Path.cwd().parent.absolute()
sys.path.insert(1, str(wdir))

from models.EEGModels import EEGNet, ShallowConvNet, DeepConvNet, EEGNet_ChanRed, CapsEEGNet, TCNet
import torch
import os

In [2]:
model = EEGNet(nb_classes = 2, Chans = 14, Samples = 128, dropoutRate = 0.1, kernLength = 16, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = 0
for name, param in model.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: block1.0.weight, Parameters: 1024
Layer: block1.1.weight, Parameters: 64
Layer: block1.1.bias, Parameters: 64
Layer: block1.2.weight, Parameters: 7168
Layer: block1.3.weight, Parameters: 512
Layer: block1.3.bias, Parameters: 512
Layer: block2.0.depthwise.weight, Parameters: 8192
Layer: block2.0.pointwise.weight, Parameters: 32768
Layer: block2.1.weight, Parameters: 64
Layer: block2.1.bias, Parameters: 64
Layer: dense.weight, Parameters: 512
Layer: dense.bias, Parameters: 2
Total Parameters: 50946


In [3]:
model_SEED = EEGNet_ChanRed(nb_classes = 3, Chans = 62, InnerChans=18, Samples = 200, dropoutRate = 0.1, kernLength = 25, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = 0
for name, param in model_SEED.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: chan_reduction.weight, Parameters: 1116
Layer: chan_reduction.bias, Parameters: 18
Layer: block1.0.weight, Parameters: 1600
Layer: block1.1.weight, Parameters: 64
Layer: block1.1.bias, Parameters: 64
Layer: block1.2.weight, Parameters: 9216
Layer: block1.3.weight, Parameters: 512
Layer: block1.3.bias, Parameters: 512
Layer: block2.0.depthwise.weight, Parameters: 8192
Layer: block2.0.pointwise.weight, Parameters: 32768
Layer: block2.1.weight, Parameters: 64
Layer: block2.1.bias, Parameters: 64
Layer: dense.weight, Parameters: 1152
Layer: dense.bias, Parameters: 3
Total Parameters: 55345


In [4]:
x = torch.randn(8, 1, 62, 200)
y = model_SEED(x)
print(y.size())

torch.Size([8, 3])


/Users/juliengadonneix/anaconda3/envs/intern3A/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1712608633180/work/aten/src/ATen/native/Convolution.cpp:1032.)
  return F.conv2d(input, weight, bias, self.stride,


In [5]:
model_capseegnet = CapsEEGNet(nb_classes=2, Chans=14)

total_params = 0
for name, param in model_capseegnet.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: block_1_2.0.weight, Parameters: 512
Layer: block_1_2.1.weight, Parameters: 8
Layer: block_1_2.1.bias, Parameters: 8
Layer: block_1_2.3.weight, Parameters: 224
Layer: block_1_2.4.weight, Parameters: 16
Layer: block_1_2.4.bias, Parameters: 16
Layer: primaryCaps.caps.weight, Parameters: 24576
Layer: primaryCaps.caps.bias, Parameters: 256
Layer: primaryCaps.caps2.weight, Parameters: 69632
Layer: primaryCaps.caps2.bias, Parameters: 256
Layer: emotionCaps.W, Parameters: 1048576
Layer: fc.weight, Parameters: 16
Layer: fc.bias, Parameters: 1
Total Parameters: 1144097


In [6]:
x = torch.randn(8, 1, 14, 128)
y = model_capseegnet(x)
print(y.size())
print(y)

torch.Size([8, 2])
tensor([[0.5009, 0.4991],
        [0.5012, 0.4988],
        [0.4994, 0.5006],
        [0.5004, 0.4996],
        [0.5010, 0.4990],
        [0.4989, 0.5011],
        [0.5001, 0.4999],
        [0.5001, 0.4999]], grad_fn=<SoftmaxBackward0>)


In [7]:
model_tcnet = TCNet(nb_classes=2, device='cpu', Chans=14)


total_params = 0
for name, param in model_tcnet.named_parameters():
    layer_params = param.numel()
    print(f"Layer: {name}, Parameters: {layer_params}")
    total_params += layer_params

print(f"Total Parameters: {total_params}")

Layer: PatchPartition.weight, Parameters: 5376
Layer: PatchPartition.bias, Parameters: 32
Layer: primaryCaps.caps.weight, Parameters: 3538944
Layer: primaryCaps.caps.bias, Parameters: 384
Layer: primaryCaps.caps2.weight, Parameters: 163840
Layer: primaryCaps.caps2.bias, Parameters: 256
Layer: emotionCaps.W, Parameters: 65536
Total Parameters: 3774368


/Users/juliengadonneix/anaconda3/envs/intern3A/lib/python3.9/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [8]:
x = torch.randn(8, 14, 48, 128)
out = model_tcnet(x)
print(out.shape)

torch.Size([8, 2])


In [9]:
print(os.cpu_count())
print(torch.cuda.device_count()) 
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(device)
print(device.type == 'cuda')
if device.type == 'cuda':
    properties = torch.cuda.get_device_properties(device)
    print(properties)

8
0
mps
False


In [10]:
import subprocess
import plistlib

def get_gpu_info():
    """Get information about the GPU on macOS."""
    # Run system_profiler command and capture the output
    result = subprocess.run(['system_profiler', '-xml', 'SPDisplaysDataType'], capture_output=True)
    if result.returncode == 0:
        # Parse the XML output
        display_info = plistlib.loads(result.stdout)
        # Extract GPU information
        gpu_count = len(display_info[0]['_items'])
        return gpu_count
    else:
        print("Error:", result.stderr)
        return None

gpu_count = get_gpu_info()
if gpu_count is not None:
    print("Number of GPUs:", gpu_count)


Number of GPUs: 1


In [11]:
import time
import torch

start_time = time.time()

# syncrocnize time with cpu, otherwise only time for oflaoding data to gpu would be measured
torch.mps.synchronize()

a = torch.ones(4000,4000, device="mps")
for _ in range(200):
   a +=a

elapsed_time = time.time() - start_time
print( "GPU Time: ", elapsed_time)

GPU Time:  0.7244799137115479


In [12]:
t = torch.randn(8,8)
print(t)
torch.reshape(t, (-1, 2, 8))

tensor([[-1.3296e+00, -1.7722e+00, -2.0000e+00, -1.2098e+00, -5.1593e-01,
         -1.3994e-01,  3.0890e-01,  1.0507e+00],
        [ 3.6133e-01,  1.3256e+00, -1.6759e-01, -1.1906e-01,  1.5129e+00,
         -6.2672e-01,  5.7583e-01, -5.0284e-01],
        [-4.0213e-01,  1.4174e+00,  1.0090e+00, -1.6705e+00, -5.1651e-01,
          3.5122e-01,  1.4479e+00, -1.0237e+00],
        [ 1.7625e-01, -1.2928e+00, -1.6349e+00, -1.1511e+00, -9.0430e-04,
          1.2438e+00,  2.3233e-01, -9.7487e-01],
        [-2.5936e-01,  7.4029e-02,  1.3072e+00, -1.3323e-01, -6.1612e-01,
          5.4580e-01, -3.4117e-01, -4.5955e-01],
        [ 1.0247e-01,  6.7699e-01, -6.6861e-01,  2.2630e-01, -2.9368e-01,
         -1.2636e+00,  1.3476e+00, -4.6094e-01],
        [-5.6880e-02,  5.1699e-01,  3.4769e-01,  9.0209e-01,  4.0056e-01,
         -1.4741e-01, -1.7207e+00, -5.1751e-01],
        [ 3.4654e-01, -6.3015e-01,  3.8711e-01, -7.0938e-01, -1.4822e+00,
          2.8392e-01,  1.9146e-01, -4.3963e-01]])


tensor([[[-1.3296e+00, -1.7722e+00, -2.0000e+00, -1.2098e+00, -5.1593e-01,
          -1.3994e-01,  3.0890e-01,  1.0507e+00],
         [ 3.6133e-01,  1.3256e+00, -1.6759e-01, -1.1906e-01,  1.5129e+00,
          -6.2672e-01,  5.7583e-01, -5.0284e-01]],

        [[-4.0213e-01,  1.4174e+00,  1.0090e+00, -1.6705e+00, -5.1651e-01,
           3.5122e-01,  1.4479e+00, -1.0237e+00],
         [ 1.7625e-01, -1.2928e+00, -1.6349e+00, -1.1511e+00, -9.0430e-04,
           1.2438e+00,  2.3233e-01, -9.7487e-01]],

        [[-2.5936e-01,  7.4029e-02,  1.3072e+00, -1.3323e-01, -6.1612e-01,
           5.4580e-01, -3.4117e-01, -4.5955e-01],
         [ 1.0247e-01,  6.7699e-01, -6.6861e-01,  2.2630e-01, -2.9368e-01,
          -1.2636e+00,  1.3476e+00, -4.6094e-01]],

        [[-5.6880e-02,  5.1699e-01,  3.4769e-01,  9.0209e-01,  4.0056e-01,
          -1.4741e-01, -1.7207e+00, -5.1751e-01],
         [ 3.4654e-01, -6.3015e-01,  3.8711e-01, -7.0938e-01, -1.4822e+00,
           2.8392e-01,  1.9146e-01, -4.3963e